<img src="http://imgur.com/1ZcRyrc.png" style="float: left; margin: 20px; height: 55px">

# Bayesian Estimation and Regression With PyMC3

_Authors: Kiefer Katovich (SF)_

---

### Learning Objectives
- Learn how to use PyMC3 to fit Bayesian estimation models.
- Run the Bayesian alternative to a t-test using PyMC3.
- Plot and interpret the results of PyMC3 models.
- Construct a Bayesian regression using PyMC3.
- Use the Patsy-style formula syntax to build a PyMC3 regression.

### Lesson Guide
- [Introduction](#intro)
- [Enter PyMC3](#pymc3)
- [Load the Starcraft Data](#data)
- [An Alternative to T-Testing Using Bayesian Estimation](#ttest)
    - [Get the APM Values for 19- and 26-Year Olds](#load)
    - [Perform a Frequentist T-Test of the Mean APM Difference Between Groups](#freq-ttest)
    - [Set Up a Bayesian Model With Priors on the Mean APMs](#priors)
    - [Construct the Prior Distributions on the Means](#construct-priors)
    - [Construct the Prior Distributions on the Standard Deviations](#std-priors)
    - [Set Up the APM Distributions for the Two Groups](#apm-dists)
    - [Tracking Additional Metrics](#metrics)
    - [Fitting the PyMC3 Model](#fit)
    - [Plotting the Posteriors and Metric Distributions](#plotting)
- [Bayesian Regression With PyMC3](#reg)
    - [Set Up Variables to Perform an Age vs. APM Regression](#age-vs-apm)
    - [Set Up the Bayesian Regression Model](#bayes-model)
    - [Plot the Posteriors Using `pm.traceplot()`](#traceplot)
- [Using a Patsy-Style Formula to Specify a PyMC3 Model](#formula)

<a id='intro'></a>
## Introduction
---

We can perform the Bayesian counterparts of the frequentist models we have studied using PyMC3. We'll start incrementally in this code-along lesson: First, we'll replace the frequentist t-test with Bayesian estimation, then we'll perform a Bayesian regression with a single predictor.


In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

plt.style.use('fivethirtyeight')

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

<a id='pymc3'></a>
## Enter PyMC3
---

PyMC3 is the Bayesian modeling package we'll be using. It’s a powerful and flexible architecture for performing a huge variety of Bayesian analyses. We will only be scratching the surface today.

> **Note:** There is also PyMC (or PyMC2), which has a different syntax. PyMC2 can be harder for beginners to Bayesian statistics to understand. PyMC3 better bridges the gap for those who are more familiar with constructing models using frequentist models and syntax.

In [2]:
import pymc3 as pm

<a id='data'></a>
### Load the Starcraft Data Set
---

You may be familiar with this data set. It has records of different player statistics in competitive Starcraft. There are a variety of columns that are described in detail in the `description.txt` file contained in the `./datasets/` folder.

We'll be using APM (actions per minute) and age for the examples in this lab, but in the following lab, you'll get the chance to look at other variables as well.

In [3]:
star = pd.read_csv('./datasets/SkillCraft1_Dataset.csv')

In [4]:
# A:

<a id='ttest'></a>
## An Alternative to T-Testing Using Bayesian Estimation
---

[This example is adapted from this documentation.](http://pymc-devs.github.io/pymc3/notebooks/BEST.html) 

> *Note: In that example, the student t-distribution is used instead of the normal distribution (which we use below). This is more appropriate but also more complicated because of the parameterization of the t-distribution. If you're feeling up for a challenge, try solving the problem below with the t-distribution like in the documentation.

In the following code, we'll be using Bayesian posterior estimation to look at the difference in APM between 19-year-old players and 26-year-old players.

### The Frequentist Strategy
In frequentist statistics, calculating the mean difference between groups is typically done with a t-test. In a t-test, we state a null hypothesis (H0) that there is no difference between groups, then evaluate the probability that we could have gotten this data when the null hypothesis is true.

The standard for statistically comparing two or more samples is to use a statistical test. This involves expressing a null hypothesis, which typically claims that there is no difference between the groups, and using a chosen test statistic to determine whether or not the distribution of the observed data is plausible under that hypothesis. This rejection occurs when the calculated test statistic is higher than some pre-specified threshold value.

### The Bayesian Strategy
The Bayesian approach for evaluating differences between groups is estimation rather than testing. Instead of asking if the two groups are different, we ask how different the two groups are. This is the subtle difference between estimation and testing. We’re measuring the extent of the difference between the groups, whether it be zero or any other value.

For a more detailed treatment, feel free to read the documentation linked above.

<a id='load'></a>
### Get the APM for 19- and 26-Year Olds

In [5]:
# A:

**Calculate the mean for the two groups and the empirical difference between the means.**

In [6]:
# A:

<a id='freq-ttest'></a>
### Perform a Frequentist T-Test of the Mean APM Difference Between Groups

What is the null hypothesis? What is the alternative hypothesis?

In [7]:
import scipy.stats as stats

In [8]:
# A:

**Plot the t-statistic for the test on the t-distribution.**

In [9]:
# A:

**What is the p value?**

In [10]:
# A:

<a id='priors'></a>
### Set Up the Bayesian Model With Priors on Mean APMs

Next, we'll walk through the steps of the Bayesian estimation of the difference between means using PyMC3.

First, we find the mean and standard deviation of APM, regardless of age group. We can use these values to inform our prior belief about APM.

In [11]:
# A:

<a id='construct-priors'></a>
### Construct the Prior Distributions on the Means

> **Note**: PyMC3 sets up models using the Python `with ... :` syntax.

The first thing we're going to set up is our prior belief about the mean APM for our 19-year olds and 26-year olds. _These are not fixed values_ but rather normal distributions covering a range of possible values for the mean APM for each group with varying likelihoods.

We are going to set up the normal distributions to take both the overall mean and standard deviation of the actions per minute for all players. In other words, our prior belief for each group's mean APM is the same.

In [12]:
# A:

<a id='std-priors'></a>
### Priors on Standard Deviations

We just set up the prior distributions for the _means_ of APM for the two age groups, but we can also set up priors on the standard deviations for APM for each group.

Our standard deviation priors can be uninformative: uniformly distributed from close to zero up to 100. We’re saying that we believe that all standard deviations in those ranges are equally likely.

In [13]:
# A:

<a id='apm-dists'></a>
### Set Up the APM Distributions for the Two Groups

Now that we have our priors about the mean and standard deviation of the APM for each age group, we can construct the posterior distributions for APMs after observing the data.

We set up a `pm.Normal` distribution to represent the posterior distributions of APM for each group. The mean `mu=` will be our prior belief distribution of the mean for that group. Likewise, the `sd=` will be the prior distribution for the standard deviation of APM for that group. 

Lastly, we provide the posteriors vectors of data for the `observed=` parameter. This is the actual, measured APM data. When we fit this model (using MCMC or another sampling strategy under the hood), the posterior distributions will be updated according to our prior beliefs and our data — just like in Bayes' theorem.

In [14]:
# A: 

<a id='metrics'></a>
### Tracking Additional Metrics

The last thing we'll do before fitting the model is define some distributions that will make it easier for us to evaluate the difference between the two groups' mean APMs. 

`pm.Deterministic` distributions are defined from other distributions in the model. For example, we can set up `diff_of_means` to be defined by `group19_mean - group26_mean`, making this a distribution of the differences of means between groups as the posteriors are iteratively fit/estimated.

We can also set up a distribution for the effect size by taking the difference between means and dividing by the pooled standard deviation.

In [15]:
# A:

<a id='fit'></a>
### Fitting the PyMC3 Model

We are starting a sampling process to estimate the posterior distribution when we call `trace = pm.sample(...)`. PyMC3 has the option to do MCMC but instead defaults to the NUTS sampler. NUTS stands for "no u-turn sampler" and is a state-of-the-art posterior estimation algorithm.

The trace is a collection of all of the values on the posterior distribution visited during the sampling procedure. The values in the trace define the posterior distribution.

In [16]:
# A:

<a id='plotting'></a>
### Plotting the Posteriors and Metric Distributions

PyMC3 comes with convenient methods for plotting posteriors. Below, we plot our posterior means and standard deviations. The relevant statistics about the distributions are automatically added to the plots.

The highest posterior density (HPD) interval gives us a Bayesian credible interval — the corollary to the frequentist confidence interval. The 95-percent HPD says that 95 percent of the highest density points on the distribution fall within that range.

**Take a look at the first element of the `trace`.**

In [17]:
# A:

**Use the `pm.plot_posterior` function to examine the posterior distributions for the group means and standard deviations.**

> **Note:** It’s common to slice off the initial portion of the trace. This is known as the burn-in. The sampling procedure often starts far away from the correct estimates, so slicing off the beginning can get rid of the cruft. It’s also common to not take every sample and instead skip every two or three. We’re not doing that here.

In [18]:
# A:

**Plot the posteriors of our metric distributions.**

In [19]:
# A:

**The `pm.summary` function can also display a text representation of this information.**

In [20]:
# A:

<a id='reg'></a>
## Bayesian Regression With PyMC3
---

The sky's the limit with PyMC3. Let's move on to something a little more complicated: Bayesian regressions.

> *Note: [This example](http://pymc-devs.github.io/pymc3/notebooks/GLM-linear.html) is modeled after the generalized linear model (GLM) instructions in the PyMC3 documentation.*

What are the benefits of taking a Bayesian approach to regression modeling? When we perform regression with Bayesian statistics, we’ll get out posterior *distributions* on our intercept and coefficients — just like in our estimation of means per group. 

This is a big difference from the point estimates coming out of the frequentist regression; not only do we have the most likely value for the coefficients (the MAP estimates), but we can also see the range of possibilities given our prior and observed data. 

---

<a id='age-vs-apm'></a>
### Set Up Variables to Perform an Age vs. APM Regression

In [21]:
# A:

**Plot out the age versus APM values.**

In [22]:
# A:

<a id='bayes-model'></a>
### Set Up the Bayesian Regression Model

We will construct our model using the `with pm.Model() ...` syntax. 

We have three prior distributions in this case that will be updated to posteriors given our observed data:

1) `apm_std`: This will be our prior belief about the standard deviation of APM values. In this case, we're setting it to be uniform (uninformative) between `0.0001` and `1000.`
2) `intercept`: A normal distribution for the range of possible values of the intercept.
3) `age_beta`: A normal distribution for the range of possible values of the age coefficient.

We set up a `likelihood` distribution that is also normal, representing the distribution of the data (APM) given our predictor. The mean of the likelihood is defined like a regression formula: our intercept distribution, plus the age values, times our age beta distribution. We give the `apm_std` a uniform prior as the standard deviation. Lastly, the observed values, or data, will be the APM values.

The more likely values (as defined by the `mu=` regression-style definition) will be visited more often as we sample from the posterior. The intercept and age_beta distributions will have more values visited in places where they result in higher likelihood estimates.

`pm.find_MAP()` tries to find good starting values for our sampling procedure so that we don't have to run it for as many iterations to get a good posterior representation.

In [23]:
# A:

<a id='traceplot'></a>
### Plot the Posteriors Using `pm.traceplot()`

The `pm.traceplot()` function can also plot our posteriors, as well as a graph of the actual traces for each distribution. These traces represent the points visited by the sampling algorithm.

In [24]:
# A:

<a id='formula'></a>
## Setting Up a Model Using a Patsy-Style Formula
---

PyMC3 also provides a convenience function — `pm.glm.glm(...` — that takes a Patsy-style formula definition and the DataFrame and automatically constructs the distributions required to solve the Bayesian regression.

In [25]:
# A:

**Lastly, you can plot out different possible regression lines with `pm.glm.plot_posterior_predictive()`.**

This function takes:

- The trace (MCMC samples).
- The number of samples you want to draw out and plot.
- An `lm=` argument that will be a function defining how the regression line will be fit using an `x` value and the sample.
- An `eval=` argument that will be the `x` values `lm` is evaluated over.

In [26]:
# A: